In [1]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt

# Additional Libraries
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns, base_optimizer

# Included in Python
import random
from datetime import datetime
from typing import Tuple, List  # For documentation
import time

Required libraries:
- pandas
- numpy
- numpy_financial
- yfinance
- matplotlib
- pypfopt

To install pypfopt, type
```
pip install PyPortfolioOpt
```

## Group Assignment
### Team Number: 02
### Team Member Names: Jason, Patrick, Gateek
### Team Strategy Chosen: Market Beat

Disclose any use of AI for this assignment below (detail where and how you used it).  Please see the course outline for acceptable uses of AI.


### STEP 1: FILTER STOCKS FOR VALID TICKERS BASED ON SET REQUIREMENTS

In [2]:
def valid_stocks(tickers_file):
    """
    Filters and returns a list of valid stock tickers based on specific restrictions:
    - The stock must trade in USD or CAD.
    - The stock must have an average monthly volume above 100,000 shares.
    - The stock must have at least 18 valid trading days per month.

    Parameters:
        tickers_file (str): Path to a CSV file containing stock tickers under a column named 'Tickers'.

    Returns:
        list of str: A list of valid stock tickers that meet the specified criteria.

    Example:
        valid_tickers = valid_stocks('Tickers_Example.csv')
    """

    # Read CSV and extract tickers
    tickers_df = pd.read_csv(tickers_file)

    if tickers_df.empty:  # If the file is empty, return nothing
        return []

    # Rename columns to ensure consistency
    tickers_df.columns = ['Tickers']
    tickers_list = tickers_df['Tickers'].tolist()

    # Define the date range for historical data
    start = '2023-10-01'
    end = '2024-09-30'

    valid_tickers = []  # List to store valid tickers

    # Process each ticker in the list
    for ticker in tickers_list:
        # Load ticker info using yfinance
        stock = yf.Ticker(ticker)
        info = stock.fast_info

        # Filter ticker by currency
        try:
            currency = info['currency']
        except Exception:  # Skip ticker if currency info is unavailable
            continue

        if currency not in ['USD', 'CAD']:  # Only allow USD or CAD stocks
            continue

        # Filter ticker by average monthly volume and trading days
        try:
            hist = stock.history(start=start, end=end, interval='1d')
        except Exception:  # Skip ticker if historical data is unavailable
            continue

        # Calculate monthly trading statistics
        monthly_volume = pd.DataFrame()
        monthly_volume['volume'] = hist['Volume'].resample('ME').sum()
        monthly_volume['count'] = hist['Volume'].resample('ME').count()
        monthly_volume['avg monthly volume'] = monthly_volume['volume'] / monthly_volume['count']

        # Check for months with insufficient trading days or low average volume
        invalid_trading_days = monthly_volume[monthly_volume['count'] < 18]
        invalid_monthly_vol = monthly_volume[monthly_volume['avg monthly volume'] < 100000]

        # Skip tickers that fail the volume or trading day criteria
        if len(invalid_monthly_vol) > 0 or len(invalid_trading_days) > 0:
            continue

        # Add valid ticker to the list
        valid_tickers.append(ticker)

    return valid_tickers

# Example usage
valid_tickers = valid_stocks('Tickers_Example.csv')

$AGN: possibly delisted; no price data found  (period=5d)
$BAC: possibly delisted; no price data found  (period=5d)
$BIIB: possibly delisted; no price data found  (period=5d)
$BK: possibly delisted; no price data found  (period=5d)
$CELG: possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")
$LMT: possibly delisted; no price data found  (period=5d)
$MO: possibly delisted; no price data found  (period=5d)
$MON: possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")
$MRK: possibly delisted; no price data found  (period=5d)
$QCOM: possibly delisted; no price data found  (period=5d)
$RTN: possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")
$T.TO: possibly delisted; no price data found  (period=5d)
$TD.TO: possibly delisted; no price data found  (period=5d)
$TXN: possibly delisted; no price data found  (period=5d)
$UNH: possibly d

### STEP 2: GET CLOSE PRICES FOR ALL VALID STOCKS

In [9]:
def get_close_prices(start, end, tickers, cutoff):
    """
    Retrieves historical close prices for specified stocks and converts them to CAD.
    Filters out stocks that do not have data available before the cutoff date. The returned 
    data starts from the date when the youngest valid stock began tracking close prices.

    Parameters:
        start (str): The start date for fetching historical data (format: 'YYYY-MM-DD').
        end (str): The end date for fetching historical data (format: 'YYYY-MM-DD').
        tickers (list of str): A list of stock ticker symbols to fetch data for.
        cutoff (str): The cutoff date (format: 'YYYY-MM-DD'). Stocks without data before this date are excluded.

    Returns:
        pd.DataFrame: A DataFrame containing the close prices of valid stocks (in CAD), starting from
                      the earliest available date for all valid stocks.

    Restrictions:
        - `start` must be less than `cutoff`, and `cutoff` must be less than `end`.

    Example:
        close_prices = get_close_prices('2020-01-01', '2024-01-01', ['AAPL', 'NVDA'], '2022-01-01')
    """

    multi_data = pd.DataFrame()  # Final DataFrame to hold valid tickers
    df = []  # Temporary list to hold individual stock close price series
    appended_tickers = []  # List of valid tickers that meet the cutoff condition

    # Loop through tickers
    for ticker in tickers:
        # Get historical data for the ticker
        data = yf.download(ticker, start=start, end=end, interval='1d')
        close = data['Close']
       #close = close.rename(ticker)  # Rename series to match the ticker symbol

        # Check if the stock has data before the cutoff date
        if close.index.min() < pd.Timestamp(cutoff):
            # Add the stock's close prices to the list
            df.append(close)
            appended_tickers.append(ticker)

    # Combine all valid stock close prices into a single DataFrame
    multi_data = pd.concat(df, axis=1)

    # Drop rows with missing data to ensure consistent data points for all tickers
    multi_data.dropna(subset=appended_tickers, inplace=True)

    # Get CAD to USD exchange rate data
    cadusd = yf.download('CAD=X', start=start, end=end, interval='1d')
    aligned_cadusd = cadusd.reindex(multi_data.index, method='ffill')
    aligned_cadusd_close = aligned_cadusd['Close'].reindex(multi_data.index)

    # Convert all stock prices to CAD
    for ticker in appended_tickers:
        stock = yf.Ticker(ticker)
        info = stock.fast_info

        # Check the currency of the stock
        currency = info['currency']
        if currency == 'USD':  # If the stock is in USD, convert it to CAD
            combined = pd.concat([multi_data[ticker], aligned_cadusd_close], axis=1).dropna()
            # convert USD to CAD
            multi_data[ticker] = combined.iloc[:, 0] * combined.iloc[:, 1]
            #multi_data[ticker] = multi_data[ticker] * aligned_cadusd['Close']
            break
    
    return multi_data

# Example usage
start = '2016-01-01'
end = '2024-11-22'
cutoff = '2019-01-01'
close_prices = get_close_prices(start, end, valid_tickers, cutoff)

display(close_prices)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['BA']: ReadTimeout(ReadTimeoutError("HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Read timed out. (read timeout=10)"))
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed dow

Ticker,ABBV,ABT,ACN,AIG,AMZN,AXP,BB.TO,BLK,BMY,C,CL,LLY,PG,SHOP.TO,USB
Date,,,,,,,,,,,,,,,
2016-01-04,79.873388,42.930000,101.830002,60.430000,31.849501,67.589996,12.75,333.100006,67.029999,51.130001,64.709999,82.870003,78.370003,3.592000,41.480000
2016-01-05,79.941077,42.919998,102.360001,60.639999,31.689501,66.550003,12.38,333.959991,68.349998,50.860001,65.000000,84.110001,78.620003,3.545000,41.680000
2016-01-06,80.240191,42.560001,102.160004,59.750000,31.632500,64.419998,12.34,330.160004,67.150002,50.119999,64.139999,83.580002,77.860001,3.561000,41.049999
2016-01-07,80.480737,41.540001,99.160004,58.320000,30.396999,63.840000,11.24,315.730011,65.290001,47.560001,63.009998,81.410004,77.180000,3.476000,40.270000
2016-01-08,78.477632,40.669998,98.199997,57.450001,30.352501,63.630001,10.93,307.779999,63.630001,46.130001,62.549999,81.250000,75.970001,3.515000,39.700001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-15,232.000693,115.900002,353.570007,75.769997,202.610001,286.869995,3.33,1047.369995,56.220001,68.760002,93.559998,746.200012,169.539993,152.869995,49.900002
2024-11-18,234.220337,117.360001,352.790009,76.309998,201.699997,285.540009,3.37,1049.969971,56.799999,69.000000,94.620003,727.200012,170.750000,148.490005,50.610001
2024-11-19,233.571133,117.129997,353.950012,75.300003,204.610001,285.549988,3.27,1028.109985,58.230000,68.599998,93.610001,729.729980,170.759995,145.990005,50.400002


### STEP 3: REMOVES POOR PERFORMING STOCKS

In [ ]:
def keep_tickers(dataframe, list):
    """
    This function, given a dataframe and list of tickers, will keep
    all items in the dataframe with a ticker in the list. Tickers must be in
    the Dataframe

    :param dataframe: pd.DataFrame
    :param list: list[Str]
    :return: pd.DataFrame
    """

    newframe = pd.DataFrame()

    for i in list:
        newframe[i] = dataframe[i]

    return newframe

In [ ]:
def stock_df_to_ticker(dataframe):
    """
        This function, when given a dataframe of stocks, will return a list of ticker strings

        :param dataframe: dataframe
        :return: list[str]
        """

    ticker_list = []

    for i in dataframe.index:
        ticker_list.append(i)

    return ticker_list

In [ ]:
def correlation_filter(prices: pd.DataFrame, max_corr: float):
    """
    This function, when given a dataframe of prices and

    :param prices: pd.DataFrame
    :param max_corr: float
    :return: list[Str]
    """
    correlations = prices.corr()

    tickers = []

    corr_list = correlations.index

    avg_corr_df = pd.DataFrame(columns = ['Correlation'])

     #add to a dataframe

    for i in corr_list:
        avg_corr_df.loc[i, 'Correlation'] = correlations.loc[i].mean()

    avg_corr_df = avg_corr_df.sort_values('Correlation', ascending = False)

    #display(avg_corr_df)

    corr_list = avg_corr_df.index

    i = 0
    
    #filter - checks the stocks near the beginning, so 
    while i < len(avg_corr_df.index):
        index = avg_corr_df.index[i]
        tick = corr_list[i]
        
        if avg_corr_df.loc[tick, 'Correlation'] <= max_corr or len(corr_list) - i <= 12:
            tickers.append(tick)
            i += 1
        else:
            print(tick + " was removed since its correlation with other stocks was too high. (" 
                  + str(np.round(avg_corr_df.loc[tick, 'Correlation'], 2)) + ")")
            i += 1
    return tickers

stock_pct_change = close_prices.pct_change(fill_method=None)
stock_pct_change.drop(index=stock_pct_change.index[0], inplace=True)

stock_pct_change = keep_tickers(stock_pct_change, correlation_filter(stock_pct_change, 0.5))

display(stock_pct_change.head())

,BLK,C,BAC,ACN,RY.TO,AXP,USB,TD.TO,UNP,TXN,...,PFE,MRK,MO,ABBV,AMZN,BMY,LLY,BIIB,SHOP.TO,BB.TO
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-05,0.002582,-0.005281,0.000000,0.005205,-0.003012,-0.015387,0.004822,-0.003909,-0.009624,-0.010490,...,0.007199,0.012767,0.020213,0.000847,-0.005024,0.019693,0.014963,0.007060,-0.013085,-0.029020
2016-01-06,-0.011379,-0.014550,-0.021303,-0.001954,-0.017166,-0.032006,-0.015115,-0.024855,-0.043217,-0.006509,...,-0.017713,-0.013735,0.010589,0.003742,-0.001799,-0.017557,-0.006301,-0.010179,0.004513,-0.003231
2016-01-07,-0.043706,-0.051077,-0.036070,-0.029366,-0.018443,-0.009003,-0.019001,-0.022039,-0.023386,-0.032198,...,-0.006643,-0.008775,-0.017407,0.002998,-0.039058,-0.027699,-0.025963,-0.032927,-0.023870,-0.089141
2016-01-08,-0.025180,-0.030067,-0.019355,-0.009681,-0.002562,-0.003289,-0.014154,0.000392,0.010673,-0.025919,...,-0.012739,-0.016936,0.001720,-0.024889,-0.001464,-0.025425,-0.001965,-0.012042,0.011220,-0.027580
2016-01-11,0.001755,0.015608,0.007237,0.010489,0.004852,0.006601,0.004030,0.006660,0.012862,0.016878,...,0.002258,0.003328,0.020948,-0.026176,0.017610,-0.010215,-0.012062,-0.007306,-0.035846,-0.023788


The first round of elimination is identifying the stocks that have too high of an average correlation with the other stocks in the portfolio. So, as discussed in our Portfolio unit, we want stocks that are diversified, so we do not want to hold stocks that have too similar of a correlation with the rest of the portfolio. If the stocks in our portfolio are too closely correlated, it could result in one event causing our entire portfolio to lose value, so by having lowly correlated stocks, we can prevent that. 

In [ ]:
#feed pct_change() data
def sort_by_sharpe(price_pct, min_sharpe, min_return, max_std):
    """
        This function, when given a dataframe of
        price percent change, a dataframe of prices, a minimum sharpe ratio, a minimum return
        and maximum standard deviation will produce a dataframe with the returns, standard deviation
        sharpe ratio and prices of all stocks in the given dataframe that meets the set criteria

        :param price_pct: dataframe
        :param min_sharpe: float
        :param min_return: float
        :param: max_std: float
        :return: dataframe
        """

    sharpe_df = pd.DataFrame(columns=['Returns', 'Std', 'Sharpe'])
    returns = 0
    std = 0
    ticker = ""

    stock_info = {}#fixed length list of Ticker, Returns, Std, Sharpe

    tick_list = price_pct.columns

    for i in range(len(tick_list)):
        ticker = tick_list[i]

        #work out equations - pct_change() on all,
        returns = price_pct[ticker].mean()

        #work out equations
        std = price_pct[ticker].std()

        #multiply sharpe_ratio by sqrt(15), or the number of weeks
        sharpe = returns/std * (252 ** 0.5)

        #print(ticker, sharpe, returns, std)
        stock_info = {
            'Returns': returns,
            'Std': std,
            'Sharpe': sharpe
        }

        # stock_info = pd.DataFrame(stock_info)

        sharpe_df.loc[ticker] = stock_info

    sharpe_df = sharpe_df.sort_values('Sharpe', ascending = False)

    #print(sharpe_df)

    #filter out the bad stocks from after the last 12 stocks, this ensures that only the worse stocks get removed. 

    i = 12

    while i < len(sharpe_df.index):
        index = sharpe_df.index[i]
        
        if sharpe_df.loc[index, 'Returns'] < min_return :
            print(index + " was removed because its returns were too low. (", np.round(sharpe_df.loc[index, 'Returns'], 4), ")")
            sharpe_df.drop(index=index, inplace=True)
        elif sharpe_df.loc[index, 'Std'] > max_std:
            print(index + " was removed because its risk was too high. (", np.round(sharpe_df.loc[index, 'Std'], 4), ")")
            sharpe_df.drop(index=index, inplace=True)
        elif sharpe_df.loc[index, 'Sharpe'] < min_sharpe:
            print(index + " was removed because its Sharpe ratio was too low. (", np.round(sharpe_df.loc[index, 'Sharpe'], 4), ")")
            sharpe_df.drop(index=index, inplace=True)
            
        else:
            i+= 1
        
        

    #print(sharpe_df)

    return sharpe_df

stock_sharpe = sort_by_sharpe(stock_pct_change, 0.1, 0, 1)
display(stock_sharpe)

BMY was removed because its Sharpe ratio was too low. ( 0.0625 )
BIIB was removed because its Sharpe ratio was too low. ( 0.0376 )
PFE was removed because its Sharpe ratio was too low. ( 0.0315 )
BB.TO was removed because its Sharpe ratio was too low. ( 0.014 )


,Returns,Std,Sharpe
SHOP.TO,0.002351,0.036142,1.032727
LLY,0.001164,0.017919,1.031311
UNH,0.000883,0.016538,0.847873
CAT,0.000984,0.019295,0.809222
AMZN,0.001051,0.020805,0.802045
ACN,0.000707,0.016060,0.698404
AXP,0.000872,0.020222,0.684219
TXN,0.000754,0.018091,0.662005
RY.TO,0.000463,0.011368,0.646093
UNP,0.000639,0.016342,0.621039


At this step of algorithm, we sort a dataframe by their sharpe values and remove any stock whose returns or sharpe values are too low. This is to ensure that the stocks that we have are a good investment. Additionally, stocks that have too high of a risk might be removed as well. Essentially, we want to have stocks that provide positive returns and meet our criteria for our stocks.

In the event that there at not 12 stocks that meet our criteria, we take the 12 that best meet our criteria.

In [ ]:

def arrange_by_sharpe(prices:pd.DataFrame, sharpe:pd.DataFrame):
    """
    This function, when given how long the list will be and categorization of stocks,
    will return a list of stocks that we want to craft the portfolio from.
    :param prices: pd.DataFrame
    :param sharpe: pd.DataFrame
    :return: pd.DataFrame
    """
    new_prices = pd.DataFrame()

    for i in sharpe.index:
        #print(i)
        #print(prices[i])
        new_prices[i] = prices[i]

    #print(new_prices)

    return new_prices

best_stock = stock_sharpe.index[0]

print("Our best stock is " + best_stock)

stock_prices = arrange_by_sharpe(close_prices, stock_sharpe)

Our best stock is SHOP.TO


No actual filtering occurs here, we only just shrink the list of prices to only contain the list of stocks that we wish to use.

In [ ]:
def categorize(df:pd.DataFrame, pivot:str, categories:int):
    """
    Categorizes stocks into different categories depending on how correlated they are
    :param df: pd.DataFrame, must have pct data
    :param pivot: string that is in df
    :param categories: int
    :return: list[list[str]]
    """
    correlations = df.corr()
    lin_space = np.linspace(0, 0.9999999, categories + 1)[1:]
    # The 0.999999 prevents adding the stock itself into the list
    stock_categories = []

    for i in range(categories):
        stock_categories.append([])

    for stock in correlations.index:

        correlation = correlations.at[pivot, stock]
        inserted = False
        i = 0
        while i < categories and not inserted:
            if correlation <= lin_space[i]:
                stock_categories[i].append(stock)
                inserted = True
            i = i + 1
    return stock_categories

stock_correlation_tiers = categorize(stock_prices, best_stock, 10)
display(stock_correlation_tiers)

[['PM', 'C', 'AIG', 'USB', 'BA', 'MO'],
 [],
 [],
 [],
 ['LLY', 'ABBV', 'LMT', 'MRK', 'KO', 'TD.TO', 'T.TO'],
 ['UNH', 'CAT', 'AXP', 'RY.TO', 'PEP', 'CL'],
 ['BAC', 'UPS'],
 ['ACN', 'TXN', 'UNP', 'PG', 'QCOM'],
 ['AMZN', 'ABT', 'BLK', 'PYPL'],
 []]

Next, we categorize the stocks we wish to build our portfolio from based on how closely correlated they are to our best stock. Additionally, each subcategory is sorted by Sharpe ratio. This prepares our program to select the best stocks so that the stocks contained in the portfolio are not too highly correlated with the other stocks, and that the stocks with higher sharpe ratios are chosen before stocks with lower sharpe ratios.

In [ ]:
def filtering(list_len, stock_correlation_tiers):
    """
    This function, when given how long the list will be and categorization of stocks,
    will return a list of stocks that we want to craft the portfolio from.
    :param list_len: int
    :param stock_correlation_tiers: list[list[str]]
    :return: list[str]
    """
    ticker_list = []

    while 0 < len(stock_correlation_tiers) and len(ticker_list) < list_len:
        sub_list = stock_correlation_tiers[0]
        while 0 < len(sub_list) and len(ticker_list) < list_len:
            #print(sub_list[0])
            ticker_list.append(sub_list[0])
            sub_list.pop(0)

        stock_correlation_tiers.pop(0)
    
    '''
    while len(ticker_list) < list_len:
        for i in range(len(stock_correlation_tiers)):
            sub_list = stock_correlation_tiers[i]

            if len(sub_list) > 0:
                ticker_list.append(sub_list[0])
                sub_list.pop(0)
                stock_correlation_tiers[i] = sub_list
    '''

    return ticker_list


ticker_lst = [best_stock]

ticker_lst += filtering(24, stock_correlation_tiers)

print(ticker_lst)

['SHOP.TO', 'PM', 'C', 'AIG', 'USB', 'BA', 'MO', 'LLY', 'ABBV', 'LMT', 'MRK', 'KO', 'TD.TO', 'T.TO', 'UNH', 'CAT', 'AXP', 'RY.TO', 'PEP', 'CL', 'BAC', 'UPS', 'ACN', 'TXN', 'UNP']


In the filtering step, we cycle through each tier of stock correlation and add them to the list of stocks we wish to buy. This continues until we have 24 stocks (or less, if stocks we have are weak). 

In [ ]:


#print(stock_sharpe)
#print(stock_correlation_tiers)


stock_close_prices = keep_tickers(close_prices, ticker_lst)
display(stock_close_prices)

,SHOP.TO,PM,C,AIG,USB,BA,MO,LLY,ABBV,LMT,...,CAT,AXP,RY.TO,PEP,CL,BAC,UPS,ACN,TXN,UNP
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-04,3.592000,87.000000,51.130001,60.430000,41.480000,140.500000,57.389999,82.870003,79.873388,213.210007,...,67.989998,67.589996,73.040001,98.769997,64.709999,16.430000,94.839996,101.830002,54.340000,78.970001
2016-01-05,3.545000,87.870003,50.860001,60.639999,41.680000,141.070007,58.549999,84.110001,79.941077,218.259995,...,67.279999,66.550003,72.820000,99.449997,65.000000,16.430000,95.779999,102.360001,53.770000,78.209999
2016-01-06,3.561000,87.849998,50.119999,59.750000,41.049999,138.830002,59.169998,83.580002,80.240191,217.630005,...,66.220001,64.419998,71.570000,99.480003,64.139999,16.080000,94.419998,102.160004,53.419998,74.830002
2016-01-07,3.476000,87.250000,47.560001,58.320000,40.270000,133.009995,58.139999,81.410004,80.480737,213.289993,...,63.939999,63.840000,70.250000,97.570000,63.009998,15.500000,92.599998,99.160004,51.700001,73.080002
2016-01-08,3.515000,86.709999,46.130001,57.450001,39.700001,129.990005,58.240002,81.250000,78.477632,212.910004,...,63.290001,63.630001,70.070000,97.209999,62.549999,15.200000,91.389999,98.199997,50.360001,73.860001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-15,152.869995,128.589996,68.760002,75.769997,49.900002,140.190002,55.930000,746.200012,232.000693,534.830017,...,384.070007,286.869995,171.130005,158.619995,93.559998,46.750000,134.070007,353.570007,201.119995,235.570007
2024-11-18,148.490005,131.720001,69.000000,76.309998,50.610001,143.869995,56.340000,727.200012,234.220337,530.960022,...,384.459991,285.540009,171.600006,158.330002,94.620003,46.720001,135.130005,352.790009,207.179993,234.410004
2024-11-19,145.990005,129.539993,68.599998,75.300003,50.400002,145.600006,55.860001,729.729980,233.571133,533.260010,...,382.329987,285.549988,171.690002,156.720001,93.610001,46.410000,133.199997,353.950012,201.070007,233.289993


Finally, reduce the number of stock prices that we have in our dataframe to contain the close prices of only the stocks we wish to keep.

# Finding the Weightings
In this section, we will try to find the optimal weightings with the stocks that have been chosen. We will be implementing ideas from Modern Portfolio Theory. In the next section, we will go over the theory behind this model. Then, we will implement our ideas to create a portfolio that meets our needs.

In [ ]:
# THIS WILL PROBABLY BE REMOVED
# This code block is to get stocks data
import jason
import markowitz
start_date = "2012-11-09"
end_date = "2024-11-09"
valid_stocks = ["AAPL", "MSFT", "AMZN", "NASDAQ", "SPY"]
close_prices = markowitz.getClosePrices(start_date, end_date, valid_stocks[:4], start_date)
display(close_prices.head())

ModuleNotFoundError: No module named 'Tools'

## Explanation
The main assumption behind the Markowitz Model is that investors are willing to accept more risk if they get more returns. In other words, the best portfolio is the one that provides the most returns for the same amount of risk. 

Firstly, we will plot out a portfolio that assigns random weightings to random stocks. We plot the returns on a scatter graph.

Note that the following code is only an example and is not coded to be dynamic. We include a dynamic implementation in the section after.

The following functions will help simulate the randomly-weighted portfolio. The first function is to generate a list of weightings that add up to one. The second function will return a dataframe with those weightings applied. The third function gets the risk and returns of these random portfolios. The fourth function will plot it out.

In [ ]:
def getRandomWeightings(length: int, min_weight: float = 0) -> List[float]:
    # Returns a list of random weightings
    # Requires that the min_weight * len <= 1
    weightings_lst = np.random.random(size=length)  # Find random weightings
    # Make sure weightings_lst sums up to weight remainder
    weightings_lst /= np.sum(weightings_lst)
    weightings_lst *= 1 - min_weight
    weightings_lst += min_weight
    return weightings_lst.tolist()
# Example:
print("Test random weightings:", getRandomWeightings(len(close_prices.columns), 0))


def applyWeightings(df: pd.DataFrame, weightings: list, investment: int) -> pd.DataFrame:
    # Returns the dataframe adjusted for all the weightings
    # Requires that df has the same number of rows as the length of weightings
    i = 0
    for column in df.columns.values:
        # find shares
        shares = investment / df[column].iloc[0]
        df[column] *= shares * weightings[i]
        i = i + 1
    return df
# Example
display(applyWeightings(close_prices, getRandomWeightings(len(close_prices.columns)), 1000000).head())


def simulateRandom(tests: int, stock_data: pd.DataFrame) -> Tuple[list, list, float, float]:
    # Simulates tests amount of tests with random weightings
    stocks_amount = len(stock_data.columns)
    results = [[], [], []]
    min_std = 1000  # We want to find the minimum and maximum standard deviations later
    max_std = -1000
    weighting_record = []
    for test in range(tests):  # simulate a set amount of tests
        weightings = getRandomWeightings(stocks_amount, 0.03)
        weighted_df = applyWeightings(stock_data, weightings, 1000000)  # Find df with weightings
        avg_return, std = markowitz.getPortfolioResults(weighted_df)  # Find metrics for performance
        results[0].append(avg_return)
        results[1].append(std)
        # Annualize Sharpe Ratio  as it is commonly meausred annualy, and 
        # we have daily data. The standard deviation does not increase 
        # at the same rate as the returns (instead, if the returns increase 
        # by x times, then the standard deviation icnreases by sqrt(x) times).
        results[2].append(avg_return / std * (252 ** 0.5))  
        min_std = min(std, min_std)
        max_std = max(std, max_std)
        weighting_record.append(weightings)
    return results, weighting_record, min_std, max_std


def plotSimulation(results: list):
    x = results[1]  # Get the risk/standard deviation
    y = results[0]  # Get the return
    colors = results[2]  # Use Sharpe Ratio to determine color
    plt.scatter(x, y, c=colors, cmap='summer')
    plt.title("Graph of Various Weightings with the Portfolio")
    plt.xlabel("Standard Deviation")
    plt.ylabel("Average Daily Returns (%)")
    plt.colorbar(label="Sharpe Ratio")


Test random weightings: [0.011774574979224262, 0.015162818750788079, 0.020644585475011343, 0.058460841712526454, 0.03781947930812088, 0.05727119371015588, 0.00718748217065714, 9.225851854949269e-05, 0.0581726824250861, 0.022512273871845585, 0.006834202541004115, 0.029430800227492653, 0.004563879893622766, 0.02988468669556647, 0.00472960358052431, 0.04700824779132781, 0.009175019635239602, 0.03445531635885007, 0.03875539177918457, 0.02192217037430653, 0.059614173849725255, 0.0034940752267977717, 0.04853686970140928, 0.05967488021225137, 0.01612708448683325, 0.028548072324081516, 0.0026677410494813023, 0.04943161301469173, 0.05112274879064228, 0.02962216000049056, 0.01628213270378952, 0.04763674790807465, 0.010029063606077719, 0.04444344100849678, 0.016911686318072813]


Ticker,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BB.TO,BIIB,...,QCOM,RY.TO,SHOP.TO,T.TO,TD.TO,TXN,UNH,UNP,UPS,USB
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-04,8561.244468,33837.743653,43519.721527,34139.681302,24415.749999,43264.907091,42591.842090,44353.618884,24356.830394,14989.148713,...,34091.808685,17852.742817,46867.921789,29732.578247,44202.581329,21079.149488,46018.375427,14716.232256,301.589330,19601.971808
2016-01-05,8568.499721,33829.859895,43746.230404,34258.319425,24293.094503,42599.198919,42764.636837,44353.618884,23650.004946,15094.972099,...,33425.209371,17798.969198,46254.673153,29865.664012,44029.784047,20858.039648,46105.307411,14574.604197,304.578519,19696.485052
2016-01-06,8600.560234,33546.107759,43660.756463,33755.517913,24249.397588,41235.765447,42085.590857,43408.775108,23573.591433,14941.324952,...,33085.107521,17493.438964,46463.439374,29513.381677,42935.417998,20722.269487,45635.086051,13944.734325,300.253744,19398.768964
2016-01-07,8626.343251,32742.135856,42378.627796,32947.645094,23302.266025,40864.503994,40321.285996,41843.035906,21472.217102,14449.352871,...,31384.598273,17170.799113,45354.370574,29529.039002,41989.158973,20055.061494,44291.598606,13618.618001,294.466181,19030.169296
2016-01-08,8411.640070,32056.393259,41968.343753,32456.142759,23268.153626,40730.081870,39405.792080,41033.170180,20880.012834,14275.355550,...,31207.746453,17126.802684,45863.237655,29294.182121,42005.616006,19535.259075,43528.975145,13763.972478,290.618407,18760.807722


Now, we will run some tests and see the results.

In [ ]:
simulation_results, simulation_weights, min_risk, max_risk = simulateRandom(500, close_prices)
plotSimulation(simulation_results)

NameError: name 'markowitz' is not defined

If all goes well, the random tests should be bounded above by a discernable line. All of the portfolios on this line represent the most efficient portfolio; in other words, they represent the portfolio with the highest return for roughly the same amount of risk. The portfolio we choose should line up on that line. We can then find the weightings by looping through our results list and finding the portfolio that matches our needs. 

However, this process is slow and unexact. Thankfully, the line, also known as the efficient frontier, that bounds the portfolios has a few special properties. Modern Portfolio Theory argues that its shape is convex, and there has been a lot of mathematical research on optimizing convex problems. We can then use the pyportfolioopt library, which gives us tools to analyze this line.

# Implementation with the Pyportfolioopt Library
To have a bit more speed and accuracy in calculating the optimal portfolio, we will be using the Pyportfolioopt library (https://pyportfolioopt.readthedocs.io/en/latest/). This library implements portfolio optimization techniques, including Markowitz. This model is effective because as we have discovered, the efficient frontier can be modeled as a convex optimization problem. 

The general outline is that we create an ```EfficientFrontier``` class from the pyportfolioopt library, give it the expected returns of the various securities and their covariance with each other, and we input the risk that we want. Then, it will solve the optimization problem. Since we do want more returns as we are trying to aim for the market beat strategy, we have made the decision to increase our risk tolerance in exchange for more returns. 

## Determining Expected Returns
The first challenge is finding a way to determine expected returns. We could use the average historical prices, but that is under the assumption that future returns will replicate previous returns. We have decided to try out the Capital Asset Pricing Model as it avoids this assumption and may be more accurate. More specifically, CAPM states that the expected return of a security can be estimated with the following formula (https://www.investopedia.com/terms/c/capm.asp#toc-capital-asset-pricing-model-capm-formula):

$$R_i = R_f + \beta(R_m - R_f)$$
Where $R_i$ denotes the expected return of the individual security, $R_f$ is the risk-free rate, and $R_m$ is the return of the market. In other words, $R_m - R_f$ is the risk premium of the Market. The idea is that the covariance with the market is a better predictor of returns than the mean historical return.

To implement CAPM, we first define the risk-free rate as $(3.31\% + 4.41\%)/2 = 3.86\%$. This is the average of the Canadian 10-year benchmark bond rate (https://www.bankofcanada.ca/rates/interest-rates/lookup-bond-yields/) and the US 10-year bond rate (https://www.bloomberg.com/markets/rates-bonds/government-bonds/us). 

The following code first gets some market data, and then finds the beta of the securities in the stocks we choose. 

In [ ]:
# Get market data
market_data = markowitz.getClosePrices(start_date, end_date, ["^GSPC"], start_date)  # I could not find the ticker for the TSX60
# It should be "TX60.TS"?
display(market_data.head())

# -- Now, we find the beta -- #
# Get stock data
stocks = markowitz.getClosePrices(start_date, end_date, valid_stocks[1:18], start_date)  # Replace WITH EXISTING STOCK DATA
beta_df = pd.concat([stocks, market_data], axis=1)
beta_df.dropna(axis=0, inplace=True)
# Find percent movement
beta_df_pct = jason.convertToPct(beta_df)
# Find covariance
market_cov = beta_df_pct.cov()["^GSPC"]
# Find market variance
market_var = beta_df_pct["^GSPC"].var()
# Find betas = cov/var
betas = market_cov / market_var
display(betas)

NameError: name 'markowitz' is not defined

Now, we want to find the CAPM for each stock. The following code first finds the market return as an average of market data. We will actually be using an exponentially weighted mean. This method allows more recent data to be weighted more heavily, since it makes sense that more recent data would be related to future data. 

To calculate exponentially weighted mean, we use the following formula:

$$a_t = \frac{c_t + c_{t-1}(1-\alpha) + c_{t-2}(1-\alpha)^2 + ... + c_0(1-\alpha)^t}{1 + (1-\alpha) + (1-\alpha)^2 + ... + (1-\alpha)^t}$$
Where $t$ is the amount of periods, $c_i$ is the percent change at time $= i$, and $\alpha$ is a factor between $0$ and $1$ that will determine how weighted previous periods are weighted. 

This is the code, which uses the pandas library.

In [ ]:
market_data_pct = jason.convertToPct(market_data)
market_data_pct.index = pd.to_datetime(market_data_pct.index)
resampled = market_data_pct.resample("M").ffill()
market_return = resampled.ewm(alpha=0.8, adjust=True).mean()["^GSPC"].iloc[-1]  # We want the last value
print("Market Return (month):", market_return)  # Remember that this is a percent, not a decimal

NameError: name 'market_data' is not defined

We can now find the expected returns using the CAPM.

In [ ]:
risk_free = 3.86
returns = risk_free/12 + betas[:-1]  * (market_return - risk_free/12) # The risk free rate is for a year, so we divide by 12
display(returns) 

NameError: name 'betas' is not defined

We now have enough information to use the pyportfolioopt library. The following code file will return the Efficient Frontier object, which we will call multiple times to get various numbers. pyportfolioopt uses a covariance matrix to determine risk, which we will store with the variable cov_matrix.

In [ ]:
def optimizedEF(returns: pd.Series, risk: pd.DataFrame, min_weight: float = 0, max_weight: float = 0.15):
    # This will return the efficient frontier (i.e. most return for different amount of risk)

    # Because there's limits, we have to incorporate them
    EF = EfficientFrontier(returns, risk, weight_bounds=(min_weight, max_weight))
    return EF

cov_matrix = jason.convertToPct(stocks).cov()
display(cov_matrix.iloc[:5, :5])

NameError: name 'stocks' is not defined

Then, we can use this to solve for various portfolios. Our hypothesis is that to get a portfolio that beats the market, we have to accept more risk, even more than the optimal Sharpe ratio accounts for. To do so, we can get the risk for the optimal sharpe ratio . Then, we can use a ratio to choose a risk value that best meets our needs, i.e. create a target risk that is higher than the risk to create the optimal sharpe ratio portfolio. The ratio that we are using is gained through our own testing.  

In [ ]:
min_weight = 1 / (2 * len(stocks.columns))  # Find the minimum weight bound

# Find sharpe ratio portfolio
sharpe_ef = optimizedEF(returns, cov_matrix, min_weight)
sharpe_weights = sharpe_ef.max_sharpe(risk_free/100)
sharpe_risk = sharpe_ef.portfolio_performance(risk_free_rate=risk_free/100)[1]

# Find target risk
target_risk = sharpe_risk * 1.1
target_ef = optimizedEF(returns, cov_matrix, min_weight)
target_weights = target_ef.efficient_risk(target_risk)
wanted_weights = target_ef.clean_weights()
target_performance = target_ef.portfolio_performance(risk_free_rate=risk_free/100)

# Display results
print(wanted_weights)

NameError: name 'stocks' is not defined

### SECOND LAST STEP: BUY SHARES AND GENERATE PORTFOLIO

##### The following function determines the currency of each stock in our portfolio 

In [ ]:
def get_currency(tickers):
    currencies = []

    for ticker in tickers:
        stock = yf.Ticker(ticker)
        info = stock.fast_info
        try:
            currency = info['currency']
        except:
            continue

        currencies.append({'Ticker': ticker, 'Currency': currency})
        
    df = pd.DataFrame(currencies)
    df.set_index('Ticker', inplace=True)

    return df

currencies = get_currency(valid_tickers)

$ACN: possibly delisted; no price data found  (period=5d)
$AIG: possibly delisted; no price data found  (period=5d)
$C: possibly delisted; no price data found  (period=5d)
$CL: possibly delisted; no price data found  (period=5d)
$PG: possibly delisted; no price data found  (period=5d)
$RY.TO: possibly delisted; no price data found  (period=5d)
$SHOP.TO: possibly delisted; no price data found  (period=5d)
$UNP: possibly delisted; no price data found  (period=5d)


##### The following function buys our stocks based on the determined weightings 

In [ ]:
def buy_shares(weightings_df, prices_df, currencies_df):
    """
    Allocates a fixed cash amount to a portfolio of stocks based on their target weightings, 
    considering transaction fees, and returns a final portfolio with adjusted investments.

    Parameters:
        weightings_df (pd.DataFrame): DataFrame with stock tickers as index and a column `Weight` representing 
                                      the target percentage allocation for each stock.
        prices_df (pd.Series): Series with stock tickers as index and their respective close prices.
        currencies_df (pd.DataFrame): DataFrame with stock tickers as index and a `Currency` column 
                                       indicating the trading currency of each stock.

    Returns:
        pd.DataFrame: A DataFrame containing the final portfolio allocation:
                      - Ticker: Stock tickers
                      - Price: Closing price of each stock
                      - Currency: Trading currency of each stock
                      - Shares: Adjusted number of shares purchased for each stock
                      - Value: Adjusted investment amount for each stock
                      - Weight: Target weight percentage for each stock

    Example:
        Portfolio_Final = buy_shares(weightings_df, close_prices.iloc[-1], currencies)
    """

    # Constants for cash amount and transaction fees
    cash = 1000000  # Total cash available for investment
    flat_fee = 3.95  # Flat transaction fee per stock
    fee_per_share = 0.001  # Fee per share

    # 1: Retrieve close prices and calculate initial investment amounts and shares
    weightings_df['Close Price'] = prices_df.reindex(weightings_df.index)
    weightings_df['Investment Amt'] = cash * (weightings_df['Weight'] / 100)
    weightings_df['Shares'] = weightings_df['Investment Amt'] / weightings_df['Close Price']

    # 2: Calculate transaction fees, choosing the cheaper of the flat fee or per-share fee
    weightings_df['fees'] = np.minimum(weightings_df['Shares'] * fee_per_share, flat_fee)

    # 3: Calculate total investment including fees
    weightings_df['Investment with fees'] = weightings_df['Shares'] * weightings_df['Close Price'] + weightings_df['fees']
    total_with_fees = weightings_df['Investment with fees'].sum()

    # 4: Adjust investments to stay within the cash budget
    adjustment_factor = cash / total_with_fees
    weightings_df['Adjusted Investment Amt'] = weightings_df['Investment Amt'] * adjustment_factor
    weightings_df['Adjusted Shares'] = weightings_df['Adjusted Investment Amt'] / weightings_df['Close Price']

    # 5: Recalculate fees based on adjusted shares
    weightings_df['Adjusted fees'] = np.minimum(weightings_df['Adjusted Shares'] * fee_per_share, flat_fee)

    # 6: Calculate final investment for each stock
    weightings_df['Final Investment'] = weightings_df['Adjusted Shares'] * weightings_df['Close Price'] + weightings_df['Adjusted fees']

    # 7: Create the final portfolio DataFrame
    Portfolio_Final = pd.DataFrame()
    Portfolio_Final['Ticker'] = weightings_df.index
    Portfolio_Final.index = Portfolio_Final['Ticker']  # Use tickers as the index
    Portfolio_Final['Price'] = weightings_df['Close Price']
    Portfolio_Final['Currency'] = currencies_df.reindex(Portfolio_Final.index)['Currency']  # Match currencies to tickers
    Portfolio_Final['Shares'] = weightings_df['Adjusted Shares']
    Portfolio_Final['Value'] = weightings_df['Adjusted Investment Amt']
    Portfolio_Final['Weight'] = weightings_df['Weight']

    # Reindex the final portfolio for cleaner display
    Portfolio_Final.index = range(1, len(Portfolio_Final) + 1)

    return Portfolio_Final


# Generate the final portfolio
Portfolio_Final = buy_shares(weightings_df, close_prices.iloc[-1], currencies)


/var/folders/7f/785d69cd4r94q3p9ft0h1nxr0000gn/T/ipykernel_1416/894985499.py:44: RuntimeWarning: divide by zero encountered in scalar divide
  adjustment_factor = cash / total_with_fees


,Ticker,Price,Currency,Shares,Value,Weight
1,USD,NaN,NaN,NaN,inf,6.67
2,USD,NaN,NaN,NaN,inf,6.67
3,USD,NaN,NaN,NaN,inf,6.67
4,USD,NaN,NaN,NaN,inf,6.67
5,USD,NaN,NaN,NaN,inf,6.67
6,CAD,NaN,NaN,NaN,inf,6.67
7,USD,NaN,NaN,NaN,inf,6.67
8,USD,NaN,NaN,NaN,inf,6.67
9,USD,NaN,NaN,NaN,inf,6.67
10,USD,NaN,NaN,NaN,inf,6.67


### TEST

In [ ]:
#tests
total = Portfolio_Final['Value'].sum()
total_weight = Portfolio_Final['Weight'].sum()
print(total, total_weight)

### LAST STEP: generating CSV file

In [ ]:
def generate_csv(portfolio_final):
    columns = portfolio_final.columns

generate_csv(Portfolio_final)

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Gateek, Jason, Patrick.